In [1]:
import pickle
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
#ipl = pd.read_csv("data/cleaned/ipl_master.csv")
ipl = pd.read_csv(
    "data/cleaned/ipl_master.csv",
    low_memory=False
)

In [3]:
print(ipl)

        match_id  inning           batting_team                 bowling_team  \
0         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
1         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
2         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
3         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
4         335982       1  Kolkata Knight Riders  Royal Challengers Bangalore   
...          ...     ...                    ...                          ...   
260915   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260916   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260917   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260918   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   
260919   1426312       2  Kolkata Knight Riders          Sunrisers Hyderabad   

        over  ball       batter        

In [4]:
#____player_match_aggregation
df = ipl.groupby(['batter','match_id','venue','bowler','bowling_team','date',]).agg({'batsman_runs':'sum','is_wicket':'sum'}).reset_index()

In [5]:
#To Sort data chronologically
df = df.sort_values(['batter','date'])

In [6]:
print(df)

               batter  match_id                            venue  \
0      A Ashish Reddy    548346                 Wankhede Stadium   
1      A Ashish Reddy    548346                 Wankhede Stadium   
2      A Ashish Reddy    548346                 Wankhede Stadium   
3      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
...               ...       ...                              ...   
54066          Z Khan   1082595            M.Chinnaswamy Stadium   
54067          Z Khan   1082635                 Feroz Shah Kotla   
54068          Z Khan   1082635                 Feroz Shah Kotla   
54069          Z Khan   1082635                 Feroz Shah Kotla   
54070          Z Khan   1082646                 Feroz Shah Kotla   

                bowler                 bowling_team        date  batsman_runs  \
0         JEC Franklin               Mumbai Indians  2012-04-29             3   
1             MM Pate

In [7]:
#______rolling averages (form)feature
df['rolling_avg_5']=df.groupby('batter')['batsman_runs'].rolling(5).mean().reset_index(0,drop=True)
                                                          

In [8]:
print(df)

               batter  match_id                            venue  \
0      A Ashish Reddy    548346                 Wankhede Stadium   
1      A Ashish Reddy    548346                 Wankhede Stadium   
2      A Ashish Reddy    548346                 Wankhede Stadium   
3      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
...               ...       ...                              ...   
54066          Z Khan   1082595            M.Chinnaswamy Stadium   
54067          Z Khan   1082635                 Feroz Shah Kotla   
54068          Z Khan   1082635                 Feroz Shah Kotla   
54069          Z Khan   1082635                 Feroz Shah Kotla   
54070          Z Khan   1082646                 Feroz Shah Kotla   

                bowler                 bowling_team        date  batsman_runs  \
0         JEC Franklin               Mumbai Indians  2012-04-29             3   
1             MM Pate

In [9]:
#____________venue averages
df['venue_avg']=df.groupby('batter')['batsman_runs'].transform('mean')
print(df)
                

               batter  match_id                            venue  \
0      A Ashish Reddy    548346                 Wankhede Stadium   
1      A Ashish Reddy    548346                 Wankhede Stadium   
2      A Ashish Reddy    548346                 Wankhede Stadium   
3      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
...               ...       ...                              ...   
54066          Z Khan   1082595            M.Chinnaswamy Stadium   
54067          Z Khan   1082635                 Feroz Shah Kotla   
54068          Z Khan   1082635                 Feroz Shah Kotla   
54069          Z Khan   1082635                 Feroz Shah Kotla   
54070          Z Khan   1082646                 Feroz Shah Kotla   

                bowler                 bowling_team        date  batsman_runs  \
0         JEC Franklin               Mumbai Indians  2012-04-29             3   
1             MM Pate

In [10]:
#_____________opponent-specific stats(Player vs Team)
#df['pvt_avg'] = df.groupby(['batter','bowler'])['batsman_runs'].transform('mean')

#___shift(1) avoids data leakage____

df['pvt_avg'] = df.groupby(['batter','bowler'])['batsman_runs']\
                           .expanding().mean().shift(1)\
                           .reset_index(level=[0,1], drop=True)
print(df)

               batter  match_id                            venue  \
0      A Ashish Reddy    548346                 Wankhede Stadium   
1      A Ashish Reddy    548346                 Wankhede Stadium   
2      A Ashish Reddy    548346                 Wankhede Stadium   
3      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
...               ...       ...                              ...   
54066          Z Khan   1082595            M.Chinnaswamy Stadium   
54067          Z Khan   1082635                 Feroz Shah Kotla   
54068          Z Khan   1082635                 Feroz Shah Kotla   
54069          Z Khan   1082635                 Feroz Shah Kotla   
54070          Z Khan   1082646                 Feroz Shah Kotla   

                bowler                 bowling_team        date  batsman_runs  \
0         JEC Franklin               Mumbai Indians  2012-04-29             3   
1             MM Pate

In [11]:
#_____________opponent-specific stats(Player vs player(bowler)
df['pvp_avg'] = df.groupby(['batter','bowler'])['batsman_runs']\
                           .expanding().mean().shift(1)\
                           .reset_index(level=[0,1], drop=True)
print(df)

               batter  match_id                            venue  \
0      A Ashish Reddy    548346                 Wankhede Stadium   
1      A Ashish Reddy    548346                 Wankhede Stadium   
2      A Ashish Reddy    548346                 Wankhede Stadium   
3      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
...               ...       ...                              ...   
54066          Z Khan   1082595            M.Chinnaswamy Stadium   
54067          Z Khan   1082635                 Feroz Shah Kotla   
54068          Z Khan   1082635                 Feroz Shah Kotla   
54069          Z Khan   1082635                 Feroz Shah Kotla   
54070          Z Khan   1082646                 Feroz Shah Kotla   

                bowler                 bowling_team        date  batsman_runs  \
0         JEC Franklin               Mumbai Indians  2012-04-29             3   
1             MM Pate

In [21]:
df['bowler_career_wickets_avg'] = (
    df.groupby('bowler')['is_wicket']
      .expanding()
      .mean()
      .shift(1)
      .reset_index(level=0, drop=True)
)
print(df)

               batter  match_id                                      venue  \
4      A Ashish Reddy    548352            MA Chidambaram Stadium, Chepauk   
5      A Ashish Reddy    548359  Rajiv Gandhi International Stadium, Uppal   
6      A Ashish Reddy    548359  Rajiv Gandhi International Stadium, Uppal   
7      A Ashish Reddy    548359  Rajiv Gandhi International Stadium, Uppal   
8      A Ashish Reddy    548373  Rajiv Gandhi International Stadium, Uppal   
...               ...       ...                                        ...   
54040          Z Khan    392192                                  Kingsmead   
54041          Z Khan    392203                               Buffalo Park   
54045          Z Khan    419165                 Dr DY Patil Sports Academy   
54046          Z Khan    501208  Rajiv Gandhi International Stadium, Uppal   
54053          Z Khan    548324                      M Chinnaswamy Stadium   

             bowler           bowling_team        date  batsman

In [12]:
#______________career stats
df['career_avg'] = df.groupby('batter')['batsman_runs']\
                           .expanding().mean().shift(1)\
                           .reset_index(level=0, drop=True)
print(df)


               batter  match_id                            venue  \
0      A Ashish Reddy    548346                 Wankhede Stadium   
1      A Ashish Reddy    548346                 Wankhede Stadium   
2      A Ashish Reddy    548346                 Wankhede Stadium   
3      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
4      A Ashish Reddy    548352  MA Chidambaram Stadium, Chepauk   
...               ...       ...                              ...   
54066          Z Khan   1082595            M.Chinnaswamy Stadium   
54067          Z Khan   1082635                 Feroz Shah Kotla   
54068          Z Khan   1082635                 Feroz Shah Kotla   
54069          Z Khan   1082635                 Feroz Shah Kotla   
54070          Z Khan   1082646                 Feroz Shah Kotla   

                bowler                 bowling_team        date  batsman_runs  \
0         JEC Franklin               Mumbai Indians  2012-04-29             3   
1             MM Pate

In [13]:
#___________Create target label
df['next_match_runs']=df.groupby('batter')['batsman_runs'].shift(-1)
df=df.dropna()
print(df)

               batter  match_id  \
4      A Ashish Reddy    548352   
5      A Ashish Reddy    548359   
6      A Ashish Reddy    548359   
7      A Ashish Reddy    548359   
8      A Ashish Reddy    548373   
...               ...       ...   
54065          Z Khan    980993   
54066          Z Khan   1082595   
54067          Z Khan   1082635   
54068          Z Khan   1082635   
54069          Z Khan   1082635   

                                                   venue           bowler  \
4                        MA Chidambaram Stadium, Chepauk         DJ Bravo   
5              Rajiv Gandhi International Stadium, Uppal          P Awana   
6              Rajiv Gandhi International Stadium, Uppal          P Kumar   
7              Rajiv Gandhi International Stadium, Uppal        PP Chawla   
8              Rajiv Gandhi International Stadium, Uppal          SW Tait   
...                                                  ...              ...   
54065  Dr. Y.S. Rajasekhara Reddy ACA-VD

In [14]:
df['next_match_wicket'] = df.groupby('bowler')['is_wicket'].shift(-1)
df=df.dropna()
print(df)

               batter  match_id                                      venue  \
4      A Ashish Reddy    548352            MA Chidambaram Stadium, Chepauk   
5      A Ashish Reddy    548359  Rajiv Gandhi International Stadium, Uppal   
6      A Ashish Reddy    548359  Rajiv Gandhi International Stadium, Uppal   
7      A Ashish Reddy    548359  Rajiv Gandhi International Stadium, Uppal   
8      A Ashish Reddy    548373  Rajiv Gandhi International Stadium, Uppal   
...               ...       ...                                        ...   
54040          Z Khan    392192                                  Kingsmead   
54041          Z Khan    392203                               Buffalo Park   
54045          Z Khan    419165                 Dr DY Patil Sports Academy   
54046          Z Khan    501208  Rajiv Gandhi International Stadium, Uppal   
54053          Z Khan    548324                      M Chinnaswamy Stadium   

             bowler           bowling_team        date  batsman

In [15]:
## _________________Feature selection
features=df[['rolling_avg_5','venue_avg','pvt_avg','pvp_avg','career_avg']]
labels  =df[['next_match_runs','next_match_wicket']]
print("Features:")
print(features.head(5))

Features:
   rolling_avg_5  venue_avg  pvt_avg  pvp_avg  career_avg
4            2.6   5.090909      5.5      5.5    3.000000
5            2.0   5.090909      0.0      0.0    2.600000
6            3.2   5.090909      0.0      0.0    2.166667
7            2.2   5.090909      6.0      6.0    2.714286
8            3.8   5.090909      4.0      4.0    2.625000


In [23]:
# sort chronologically
df = df.sort_values(['batter', 'date'])

# features and labels
X = df[['rolling_avg_5', 'venue_avg', 'pvt_avg', 'pvp_avg', 'career_avg']]
y = df[['next_match_runs', 'next_match_wicket']]

# time-based train–test split (80–20)
split = int(len(df) * 0.8)

X_train = X.iloc[:split]
X_test  = X.iloc[split:]

y_train = y.iloc[:split]
y_test  = y.iloc[split:]
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(40426, 5) (10107, 5)
(40426, 2) (10107, 2)


In [17]:
#_________________Feature pipeline
pipeline=Pipeline([
    ('scaler',StandardScaler())
])
pipeline.fit(X_train)
joblib.dump(pipeline,"scripts/feature_pipeline.pkl")

['scripts/feature_pipeline.pkl']

In [18]:
#____________save Final Dataset
final_df = df[['batter','date','venue','bowler','bowling_team','rolling_avg_5','venue_avg','pvt_avg','pvp_avg','career_avg','next_match_runs']]
print(final_df)

               batter        date                                      venue  \
4      A Ashish Reddy  2012-05-04            MA Chidambaram Stadium, Chepauk   
5      A Ashish Reddy  2012-05-08  Rajiv Gandhi International Stadium, Uppal   
6      A Ashish Reddy  2012-05-08  Rajiv Gandhi International Stadium, Uppal   
7      A Ashish Reddy  2012-05-08  Rajiv Gandhi International Stadium, Uppal   
8      A Ashish Reddy  2012-05-18  Rajiv Gandhi International Stadium, Uppal   
...               ...         ...                                        ...   
54040          Z Khan  2009-04-25                                  Kingsmead   
54041          Z Khan  2009-05-01                               Buffalo Park   
54045          Z Khan  2010-04-25                 Dr DY Patil Sports Academy   
54046          Z Khan  2011-04-14  Rajiv Gandhi International Stadium, Uppal   
54053          Z Khan  2012-04-15                      M Chinnaswamy Stadium   

             bowler           bowling_t

In [19]:
final_df.to_csv("data/processed/dataset.csv",index=False)
print("Milestone-2 is completed:dataset.csv & feature_pipeline.pkl created!")

Milestone-2 is completed:dataset.csv & feature_pipeline.pkl created!
